In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Load data into Spark RDD

In [1]:
import urllib.request

In [2]:
f = urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/kddcup99-mld/kddcup.data.gz",
                              "kddcup.data.gz")

In [3]:
raw_data = sc.textFile("./kddcup.data.gz")

In [5]:
raw_data

./kddcup.data.gz MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

Here, it is telling us that the underlying file is located in Mark map partitions 
RDD

# Parallelization with Spark RDDs

In [8]:
# list_rdd = sc.parallelize("./kddcup.data.gz")

# Basic RDD operation

In [19]:
contains_normal = raw_data.filter(lambda line: "normal." in line)

In [20]:
contains_normal.count()

972781

In [21]:
split_file = raw_data.map(lambda line: line.split(","))

In [22]:
split_file.take(5)

[['0',
  'tcp',
  'http',
  'SF',
  '215',
  '45076',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '0',
  '0',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '162',
  '4528',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '2',
  '2',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '1',
  '1',
  '1.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  'normal.'],
 ['0',
  'tcp',
  'http',
  'SF',
  '236',
  '1228',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '1',
  '1',
  '0.00',
  '0.00',
  '0.00',
  '0.00',
  '1.00',
  '0.00',
  '0.00',
  '2',
  '2',
  '1.00',
  '0.00',
  

Here we have calculated time taken by count operation when a 10 percent of sample data is taken and when the count operatoin is applied to entire data set.

Moreover, while using small data-set it is easy find any bug.

Here we have used a filter it will only look for those line which contains normal.

In [5]:
from time import time

In [4]:
sample = raw_data.sample(False,0.1,42)

In [13]:
con_sample = sample.map(lambda x: x.split(",")).filter(lambda x: "normal" in x)

In [17]:
t0 = time()
num_samp = con_sample.count()
duration = time() - t0

In [18]:
duration

9.94949984550476

In [13]:
contains_norm = raw_data.map(lambda x: x.split(",")).filter(lambda x: "normal" in x)

In [14]:
t0 = time()
num_samp = contains_norm.count()
duration = time() - t0

In [15]:
duration

15.184983015060425

# Splitting Datasets and Creating New Combinations using Set Operations

In [7]:
normal_sample = sample.filter(lambda line: "normal." in line)

In [8]:
non_sample = sample.subtract(normal_sample)

In [10]:
sample.count()

490705

This sample contains line having normal in it 

In [29]:
normal_sample.count()

97404

This sample contains data points which does not cantain normal

In [28]:
non_sample.count()

393301

In [7]:
feature_1 = sample.map(lambda line: line.split(",")).map(lambda features: features[1]).distinct()

In [8]:
feature_2= sample.map(lambda line: line.split(",")).map(lambda features: features[2]).distinct()

In [9]:
f1 = feature_1.collect()

In [10]:
f1

['tcp', 'udp', 'icmp']

In [11]:
f2 = feature_2.collect()

In [12]:
f2

['http',
 'finger',
 'auth',
 'domain_u',
 'smtp',
 'ftp',
 'telnet',
 'eco_i',
 'ntp_u',
 'ecr_i',
 'other',
 'private',
 'pop_3',
 'ftp_data',
 'daytime',
 'remote_job',
 'supdup',
 'name',
 'ssh',
 'domain',
 'gopher',
 'time',
 'rje',
 'ctf',
 'mtp',
 'X11',
 'urp_i',
 'pm_dump',
 'IRC',
 'exec',
 'bgp',
 'nnsp',
 'iso_tsap',
 'http_443',
 'login',
 'shell',
 'printer',
 'efs',
 'courier',
 'uucp',
 'kshell',
 'klogin',
 'whois',
 'echo',
 'discard',
 'systat',
 'netstat',
 'hostnames',
 'csnet_ns',
 'pop_2',
 'sunrpc',
 'uucp_path',
 'nntp',
 'netbios_ns',
 'netbios_ssn',
 'netbios_dgm',
 'imap4',
 'sql_net',
 'vmnet',
 'link',
 'Z39_50',
 'ldap',
 'urh_i',
 'tftp_u',
 'red_i',
 'tim_i']

cartesian product finding total combination possible

In [13]:
len(feature_1.cartesian(feature_2).collect())

198

# Computation and Aggregation

In [18]:
raw_data = sc.textFile("./kddcup.data.gz")

In [19]:
csv = raw_data.map(lambda x: x.split(","))

In [20]:
norm_data = csv.filter(lambda x: x[41] == "normal")

Here we are converting the data into integer

In [21]:
duration = norm_data.map(lambda x: int(x[0]))

In [ ]:
total_duration = duration.reduce(lambda x, y: x+y)
# 21189753

In [40]:
# duration_count = duration.aggregate((0,0),(lambda db, new_value: (db[0] + new_value, db[1] + 1)),(lambda db1, db2: (db1[0] +db2[0] ,db1[1]+db2[2])))

# Pivot tabling with key value paired data points

x[41] is the key and x is the value

In [45]:
kv = csv.map(lambda x: (x[41], x))
kv.take(1)

[('normal.',
  ['0',
   'tcp',
   'http',
   'SF',
   '215',
   '45076',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '1',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '0',
   '0',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'])]

In [47]:
kv.countByKey()

defaultdict(int,
            {'normal.': 972781,
             'buffer_overflow.': 30,
             'loadmodule.': 9,
             'perl.': 3,
             'neptune.': 1072017,
             'smurf.': 2807886,
             'guess_passwd.': 53,
             'pod.': 264,
             'teardrop.': 979,
             'portsweep.': 10413,
             'ipsweep.': 12481,
             'land.': 21,
             'ftp_write.': 8,
             'back.': 2203,
             'imap.': 12,
             'satan.': 15892,
             'phf.': 4,
             'nmap.': 2316,
             'multihop.': 7,
             'warezmaster.': 20,
             'warezclient.': 1020,
             'spy.': 2,
             'rootkit.': 10})

In [42]:
kv_duration = csv.map(lambda x: (x[41],float(x[0]))).reduceByKey(lambda x, y: x + y)

In [43]:
kv_duration.collect()

[('normal.', 211895753.0),
 ('buffer_overflow.', 2751.0),
 ('loadmodule.', 326.0),
 ('perl.', 124.0),
 ('neptune.', 2.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('pod.', 0.0),
 ('teardrop.', 0.0),
 ('portsweep.', 24257982.0),
 ('ipsweep.', 13049.0),
 ('land.', 0.0),
 ('ftp_write.', 259.0),
 ('back.', 284.0),
 ('imap.', 72.0),
 ('satan.', 500.0),
 ('phf.', 18.0),
 ('nmap.', 0.0),
 ('multihop.', 1288.0),
 ('warezmaster.', 301.0),
 ('warezclient.', 627563.0),
 ('spy.', 636.0),
 ('rootkit.', 1008.0)]

# Computing Summary Statistics with MLlib

In [4]:
raw_data = sc.textFile("./kddcup.data.gz")

In [5]:
csv = raw_data.map(lambda x: x.split(","))
duration = csv.map(lambda x:[int(x[0])])
urgent = csv.map(lambda x:[int(x[8])])

In [6]:
from pyspark.mllib.stat import *

In [26]:
sum_duration = Statistics.colStats(duration)
sum_urgent = Statistics.colStats(urgent)

In [27]:
sum_duration.mean()[0]

48.34243046395863

In [28]:
sum_urgent.mean()[0]

7.961733052889793e-06

In [29]:
from math import sqrt
sqrt(sum_duration.variance()[0]) # std. dev

723.3298112546713

In [34]:
sqrt(sum_urgent.variance()[0])

0.007215083685169108

In [32]:
summary.max()

array([58329.])

In [33]:
summary.min()

array([0.])

# Using pearson to discover correlation

In [7]:
# duration, src_bytes, dst_bytes
metric = csv.map(lambda x:[x[0], x[4], x[5]])
Statistics.corr(metric, method= "pearson")

array([[1.00000000e+00, 4.12205545e-02, 2.03915936e-02],
       [4.12205545e-02, 1.00000000e+00, 2.39337570e-04],
       [2.03915936e-02, 2.39337570e-04, 1.00000000e+00]])

# Using SparkSQL for manipulating Data Frames

In [11]:
raw_data = sc.textFile("./kddcup.data.gz")

In [12]:
from pyspark.sql import Row, SQLContext

In [13]:
sql_cont = SQLContext(sc)

In [14]:
csv = raw_data.map(lambda l: l.split(","))

In [15]:
rows = csv.map(lambda p: Row(duration = int(p[0]), protocol = p[1], service = p[2]))

In [16]:
df = sql_cont.createDataFrame(rows)

In [17]:
df.registerTempTable("rdd")

In [19]:
sql_cont.sql("""Select duration from rdd where protocol ='tcp' and duration > 2000""").show()

+--------+
|duration|
+--------+
|   10217|
|   11610|
|   13724|
|   10934|
|   12026|
|    5506|
|   12381|
|    9733|
|   17932|
|   40504|
|   11565|
|   12454|
|    9473|
|   12865|
|   11288|
|   10501|
|   14479|
|   10774|
|   10007|
|   12828|
+--------+
only showing top 20 rows



In [20]:
sql_cont.sql("""Select duration from rdd where protocol ='tcp' and service ='http'""").show()

+--------+
|duration|
+--------+
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
|       0|
+--------+
only showing top 20 rows



In [21]:
sql_cont.sql("""Select count(service) from rdd where protocol ='tcp'""").show()

+--------------+
|count(service)|
+--------------+
|       1870598|
+--------------+

